In [1]:
import pandas as pd
import numpy as np

import sys
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import re
import string

import nltk as nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

pd.set_option('display.max_colwidth', None)

import bokeh
import ktrain

print('Python Version: ' + sys.version)
print('TensorFlow Version: ' + tf.__version__)

Python Version: 3.7.6 (default, Jan  8 2020, 20:23:39) [MSC v.1916 64 bit (AMD64)]
TensorFlow Version: 2.5.0


In [2]:
#Verify we got CPU + GPU or only CPU
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [2]:
df_news = pd.read_json('https://storage.googleapis.com/msca-bdp-data-open/news/news_final_project.json', orient='records', lines=True) 

In [4]:
df_news['text'].isnull().values.any()

False

In [20]:
mask = df_news['text'].str.len()  < 15

In [21]:
df_news.loc[mask]

,date,language,title,text
36,2021-06-03,english,"Cite Chicago, IL",mother’s day
59,2020-08-06,english,Campio extended patio for Game 3 vs. Chicago | 5 August 2020… | Jennie,5 August 2020.
131,2020-07-19,english,Illinois Department of Agriculture Announces Details for Junior Livestock Expo and Junior Horse Show,
294,2020-12-30,english,Illinois Judge Rules Against Craft Cannabis Growers Seeking Immediate Licenses | High Times,
445,2021-08-03,english,"Shams: Spurs Sending DeMar DeRozan to Chicago for Thad Young, a Future FRP, & 2 SRPs",LFG
...,...,...,...,...
200050,2020-07-08,english,COVID-19 ILLINOIS: 587 New Confirmed Cases Tuesday; Kane Hitting Targets,
200148,2021-04-13,english,"More Than 1,000 Illinois Vaccination Locations Now Open to Residents 16 And Older",
200356,2021-01-08,english,Illinois -7 2h,Lock it in
200582,2020-11-12,english,"Jewel, Chicago Bears, and Salvation Army Event",


In [22]:
df = df_news.loc[~mask]

In [26]:
len(df_news) 

200891

In [4]:
df_news.head()

,date,language,title,text
0,2021-07-23,english,chicago theater seating chart with seat numbers,Chicago Theatre Seating Chart With Seat Numbers Tickpick Chicago Theatre Seating Chart Theatre In Chicago 47 All Inclusive The Chicago Theater Seating Cibc Theatre Seating Chart Hamilton Seat Views Tickpick Cibc Theatre Seating Chart Theatre In Chicago Cibc Theatre Seating Chart Hamilton Seat Views Tickpick Chicago Theatre Seating Chart With Seat Numbers Tickpick Nederlander Theatre Seating Chart Theatre In Chicago The Chicago Theatre Seat Map Msg Official Site Shubert Theatre Seating Chart Best Seats Pro Tips And More Cadillac Palace Theatre Seating Chart Theatre In Chicago St James Theatre Seating Chart Frozen Guide Best Seats Chicago Theater Seating Chart Balcony Image Balcony And Winter Garden Theatre Online Charts Collection Athenaeum Theatre Main Stage Seating Chart Theatre In Spectrum Center Seating Chart With Seat Numbers Seating Chart Individual Tickets Amalie Arena Seating Chart With Rows Amalie Arena Seating Chart With Rows And Seat Numbers Chicago Theater Seating Chart Main Floor Best Of Folger Seating Chart Greek Theatre Venue Seating Charts 101 9fm The Mix Wtmx Chicago Theatre Seat Numbers Online Charts Collection 2 Tickets Hamilton 6 1 19 Cibc Theatre Chicago Il Ebay Allstate Arena Seat Row Numbers Detailed Seating Chart Lyric Opera Seating Chart Lyric Opera Of Chicago Chicago United Center Seat Numbers Detailed Seating Plan Ambassador Theatre Seating Chart Chicago Tickets Reviews 10 Things We Learned Buying Hamilton Chicago Tickets Image Result For Altria Theater Detailed Seating Chart 25 Elegant Scottrade Center Detailed Seating Chart Seaket Com Seating Chart Seat Number Spac Seating Chart Hd Png Lyric Opera Seating Chart Lyric Opera Of Chicago Seating Plan Lyric Theatre Rational The Majestic Seating Chart Chicago Theatre Seating Music Box Theatre Seating Chart Best Seats Pro Tips And 70 Clean Booth Playhouse Seating Chart Seating Chart Free Png Download Seat Number Rosemont Theater Seating Venue Seating Charts 101 9fm The Mix Wtmx Chicago 71 Best Of Image Of Warner Theatre Seating Chart The Chicago Theatre Chicago Il Seating Chart Stage Section 101 Gila River Arena Seating Chart With Seat Music Theater Seat Online Charts Collection 66 Prototypical Atlanta Hawks Arena Seating Chart Seating Chart Riviera Theatre Seating Chart And Events Schedule Systematic Seat Map Of B1 Msg Seating Chart With Seat Numbers Www Bedowntowndaytona Com Scotiabank Arena Seating Chart Rows Seat Numbers And Club The Most Amazing As Well As Lovely Staples Center Seating Chicago Theatre Main Floor 3l Rateyourseats Com
1,2021-07-24,english,harris theater chicago seating chart,Harris Theater Seatmap Chicago Opera Theater Chicago Harris Theater Seating Chart English Shen Yun Harris Theater 2019 Seating Chart Chicago Gay Mens Chorus Harris Theater Chicago Gay Mens Harris Theater Chicago Il Seating Chart Stage Chicago Gay Mens Chorus Harris Theater Chicago Gay Mens Photos At Harris Theater Harris Theater Tickets In Chicago Illinois Harris Theater Seat Map Chicago Opera Theater Harris Theater For Music And Dance Seating Chart Genuine Harris Center Seating Maps Photos At Harris Theater Harris Theater Chicago Wikipedia Harris Center Seating Maps Cibc Theatre Seating Chart Hamilton Seat Views Tickpick Harris Theater Chicago Wikipedia Hubbard Street Dance Chicago Packages Harris Theater Chicago Seating Chart Related Keywords Broadway In Chicago Wikiwand Your A To Z Guide To Broadway Theater Seating Charts Your A To Z Guide To Broadway Theater Seating Charts Map Of Millennium Park Plan Your Experience The Grant Seating Charts United Center Lyric Opera Seating Chart Lyric Opera Of Chicago Upcoming Events At The Copernicus Center In Chicago Chicago United Center View From Section 301 Row 8 Seat 15 Meticulous Young Auditorium Seating Chart Rosemont Rosemont Theatre Seating Chart English Shen Yun Millennium Park In Chicago Find Park Info Art Events Tours Bro

In [5]:
# Remove special characters to avoid problems with analysis
df_news['text_clean'] = df_news['text'].map(lambda x: re.sub('[^a-zA-Z0-9 @ . , : - _]', '', str(x)))

In [6]:
# we will use 10% of the total articles to get the topics and then train a classifier

df_sample = df_news.sample(frac = 0.1, random_state = 42)

In [7]:
#texts = df_sample['text_clean'].tolist()
texts = df_news['text_clean'].tolist()
texts[:5]

['Chicago Theatre Seating Chart With Seat Numbers Tickpick Chicago Theatre Seating Chart Theatre In Chicago 47 All Inclusive The Chicago Theater Seating Cibc Theatre Seating Chart Hamilton Seat Views Tickpick Cibc Theatre Seating Chart Theatre In Chicago Cibc Theatre Seating Chart Hamilton Seat Views Tickpick Chicago Theatre Seating Chart With Seat Numbers Tickpick Nederlander Theatre Seating Chart Theatre In Chicago The Chicago Theatre Seat Map Msg Official Site Shubert Theatre Seating Chart Best Seats Pro Tips And More Cadillac Palace Theatre Seating Chart Theatre In Chicago St James Theatre Seating Chart Frozen Guide Best Seats Chicago Theater Seating Chart Balcony Image Balcony And Winter Garden Theatre Online Charts Collection Athenaeum Theatre Main Stage Seating Chart Theatre In Spectrum Center Seating Chart With Seat Numbers Seating Chart Individual Tickets Amalie Arena Seating Chart With Rows Amalie Arena Seating Chart With Rows And Seat Numbers Chicago Theater Seating Chart Ma

In [8]:
%%time
tm = ktrain.text.get_topic_model(
    texts=texts, 
    n_topics=50, 
    n_features=10000, 
    min_df=5, 
    max_df=0.5, 
    stop_words='english', 
    model_type='lda', 
    lda_max_iter=5, 
    verbose=1)

lang: en
preprocessing texts...
fitting model...
iteration: 1 of max_iter: 5
iteration: 2 of max_iter: 5
iteration: 3 of max_iter: 5
iteration: 4 of max_iter: 5
iteration: 5 of max_iter: 5
done.
Wall time: 46min 50s


In [9]:
tm.print_topics()

topic 0 | football sports smith jackson stadium memorial field brown saturday camp
topic 1 | online com free email website information post site contact link
topic 2 | food restaurant restaurants store bar stores pizza dining menu delivery
topic 3 | illinois state illini big team game coach michigan basketball college
topic 4 | news media twitter cbs fox nbc video facebook read ago
topic 5 | hair air home repair auto extensions clean area control kitchen
topic 6 | health university care medical patients center research illinois hospital program
topic 7 | shares stock quarter tool rating illinois works company price companys
topic 8 | photos flickr gutter gutters class thompson explore bauer cleaning uploaded
topic 9 | garage sign door newsletter email southern time breaking alerts heights
topic 10 | illinois state election house vote press democratic republican associated rep
topic 11 | roofing roof commercial loan loans time just make home use
topic 12 | illinois state live southern f

In [10]:
%%time
#tm.build(texts, threshold=0.25)
tm.build(texts)

done.
Wall time: 7min 36s


In [12]:
df_news.reset_index(inplace=True)

In [13]:
def get_topics_articles(idx):
#     print(idx)
#     print([idx])
    return np.argmax(tm.get_doctopics(doc_ids=[idx]))

In [21]:
#df_news['topics'] = df_news.apply(lambda x: np.argmax(tm.get_doctopics(doc_ids=x['index']),axis=1))

#df_news.apply(lambda x: print(x['index']),axis=1)

df_news['topics'] = df_news.apply(lambda x: get_topics_articles(x['index']), axis=1)

In [ ]:
np.argmax(tm.get_doctopics(doc_ids=[0]))

In [ ]:
df_news['text_clean'].isnull().values.any()

In [ ]:
#df_news['topic'] = df_news.apply(lambda x: tm.predict(list(x['text_clean'])), axis = 1)

In [22]:
df_news.head()

,index,date,language,title,text,text_clean,topics
0,0,2021-07-23,english,chicago theater seating chart with seat numbers,Chicago Theatre Seating Chart With Seat Numbers Tickpick Chicago Theatre Seating Chart Theatre In Chicago 47 All Inclusive The Chicago Theater Seating Cibc Theatre Seating Chart Hamilton Seat Views Tickpick Cibc Theatre Seating Chart Theatre In Chicago Cibc Theatre Seating Chart Hamilton Seat Views Tickpick Chicago Theatre Seating Chart With Seat Numbers Tickpick Nederlander Theatre Seating Chart Theatre In Chicago The Chicago Theatre Seat Map Msg Official Site Shubert Theatre Seating Chart Best Seats Pro Tips And More Cadillac Palace Theatre Seating Chart Theatre In Chicago St James Theatre Seating Chart Frozen Guide Best Seats Chicago Theater Seating Chart Balcony Image Balcony And Winter Garden Theatre Online Charts Collection Athenaeum Theatre Main Stage Seating Chart Theatre In Spectrum Center Seating Chart With Seat Numbers Seating Chart Individual Tickets Amalie Arena Seating Chart With Rows Amalie Arena Seating Chart With Rows And Seat Numbers Chicago Theater Seating Chart Main Floor Best Of Folger Seating Chart Greek Theatre Venue Seating Charts 101 9fm The Mix Wtmx Chicago Theatre Seat Numbers Online Charts Collection 2 Tickets Hamilton 6 1 19 Cibc Theatre Chicago Il Ebay Allstate Arena Seat Row Numbers Detailed Seating Chart Lyric Opera Seating Chart Lyric Opera Of Chicago Chicago United Center Seat Numbers Detailed Seating Plan Ambassador Theatre Seating Chart Chicago Tickets Reviews 10 Things We Learned Buying Hamilton Chicago Tickets Image Result For Altria Theater Detailed Seating Chart 25 Elegant Scottrade Center Detailed Seating Chart Seaket Com Seating Chart Seat Number Spac Seating Chart Hd Png Lyric Opera Seating Chart Lyric Opera Of Chicago Seating Plan Lyric Theatre Rational The Majestic Seating Chart Chicago Theatre Seating Music Box Theatre Seating Chart Best Seats Pro Tips And 70 Clean Booth Playhouse Seating Chart Seating Chart Free Png Download Seat Number Rosemont Theater Seating Venue Seating Charts 101 9fm The Mix Wtmx Chicago 71 Best Of Image Of Warner Theatre Seating Chart The Chicago Theatre Chicago Il Seating Chart Stage Section 101 Gila River Arena Seating Chart With Seat Music Theater Seat Online Charts Collection 66 Prototypical Atlanta Hawks Arena Seating Chart Seating Chart Riviera Theatre Seating Chart And Events Schedule Systematic Seat Map Of B1 Msg Seating Chart With Seat Numbers Www Bedowntowndaytona Com Scotiabank Arena Seating Chart Rows Seat Numbers And Club The Most Amazing As Well As Lovely Staples Center Seating Chicago Theatre Main Floor 3l Rateyourseats Com,Chicago Theatre Seating Chart With Seat Numbers Tickpick Chicago Theatre Seating Chart Theatre In Chicago 47 All Inclusive The Chicago Theater Seating Cibc Theatre Seating Chart Hamilton Seat Views Tickpick Cibc Theatre Seating Chart Theatre In Chicago Cibc Theatre Seating Chart Hamilton Seat Views Tickpick Chicago Theatre Seating Chart With Seat Numbers Tickpick Nederlander Theatre Seating Chart Theatre In Chicago The Chicago Theatre Seat Map Msg Official Site Shubert Theatre Seating Chart Best Seats Pro Tips And More Cadillac Palace Theatre Seating Chart Theatre In Chicago St James Theatre Seating Chart Frozen Guide Best Seats Chicago Theater Seating Chart Balcony Image Balcony And Winter Garden Theatre Online Charts Collection Athenaeum Theatre Main Stage Seating Chart Theatre In Spectrum Center Seating Chart With Seat Numbers Seating Chart Individual Tickets Amalie Arena Seating Chart With Rows Amalie Arena Seating Chart With Rows And Seat Numbers Chicago Theater Seating Chart Main Floor Best Of Folger Seating Chart Greek Theatre Venue Seating Charts 101 9fm The Mix Wtmx Chicago Theatre Seat Numbers Online Charts Collection 2 Tickets Hamilton 6 1 19 Cibc Theatre Chicago Il Ebay Allstate Arena Seat Row Numbers Detailed Seating Chart Lyric Opera Seating Chart Lyric Opera Of Chicago Chicago United Cente

In [23]:
df_news.shape

(200891, 7)

In [24]:
df_news.to_csv('topic_full_data.csv')

### Basic method to only extract text with Chicago or Illinois in it

In [ ]:
df1 = df_news[df_news['text'].str.contains('Chicago') | df_news['text'].str.contains('chicago') | \
       df_news['text'].str.contains('CHICAGO') | df_news['text'].str.contains('Illinois') | \
       df_news['text'].str.contains('illinois') | df_news['text'].str.contains('ILLINOIS') & \
             ~(df_news['text'].str.contains('Weather Update') | df_news['text'].str.contains('Weather forecast'))]

In [ ]:
df1

In [ ]:
df1.text[12]